# Generates Rollout for existing Checkpoint

This notebook will demonstrate the clean usage of the `track_mjx.analysis.rollout` module, which allows user to load the checkpoint from the previous training run, and perform a rollout of the checkpoint. This module abstracted away all the boilerplate codes for initializing the environment, and it is very clean and digestible, while customizable.

### Step 1: Imports and recover the config

In [1]:
from omegaconf import OmegaConf
from track_mjx.analysis.rollout import restore_config, create_rollout_generator, create_environment, create_inference_fn

# recover the config
ckpt_path = "/Users/charleszhang/GitHub/track-mjx"  # replace with your checkpoint path
config = restore_config(ckpt_path)
cfg = OmegaConf.create(config)

# make some changes to the config
# replace with absolute path to your data
# -- your notebook may not have access to the same relative path
cfg.data_path = "/Users/charleszhang/GitHub/track-mjx/ReferenceClip.p"
cfg.train_setup.checkpoint_to_restore = ckpt_path

latest checkpoint step: 149


### Step 2: Restore policy and make rollout functions

In [2]:
env = create_environment(cfg)
inference_fn = create_inference_fn(env, cfg)
generate_rollout = create_rollout_generator(cfg["reference_config"], env, inference_fn)

/Users/charleszhang/miniforge3/envs/rl/lib/python3.11/site-packages/brax/io/mjcf.py:344: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  link = jax.tree_map(lambda x: x[1:].copy(), link)
/Users/charleszhang/miniforge3/envs/rl/lib/python3.11/site-packages/brax/io/mjcf.py:374: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  motion = jax.tree_map(lambda *x: np.concatenate(x), *motions)
/Users/charleszhang/miniforge3/envs/rl/lib/python3.11/site-packages/brax/io/mjcf.py:378: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  limit = jax.tree_map(lambda *x: np.concatenate(x), *limits)
/Users/charleszhang/miniforge3/envs/rl/lib/python3.11/site-packages/brax/io/mjcf.py:414: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (j

self._steps_for_cur_frame: 2.0
latest checkpoint step: 149


### Step 3: Generate rollouts from the checkpoint!

In [3]:
# first pass will take ~3m47s to run to compile
# after complied, it only takes ~9 seconds to run
output = generate_rollout(5)

/Users/charleszhang/GitHub/track-mjx/track_mjx/analysis/rollout.py:274: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  rollout_states = jax.tree_map(prepend, init_state, states)


In [ ]:
import jax
from jax import numpy as jp
from functools import partial
jit_vmap_generate_rollout = jax.jit(jax.vmap(generate_rollout))
clip_idxs = jp.arange(0, 5)
jit_vmap_out = jit_vmap_generate_rollout(clip_idxs)


/Users/charleszhang/GitHub/track-mjx/track_mjx/analysis/rollout.py:274: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  rollout_states = jax.tree_map(prepend, init_state, states)


In [8]:
clip_idxs = jp.arange(5, 10)
jit_vmap_out2 = jit_vmap_generate_rollout(clip_idxs)


In [10]:
clip_idxs = jp.arange(10, 15)
jit_vmap_out3 = jit_vmap_generate_rollout(clip_idxs)

### Step 4: Save it to disk

In [5]:
from track_mjx.analysis.utils import save_to_h5py, load_from_h5py
import h5py

In [6]:
with h5py.File("rodent_clip_2.h5", "w") as h5file:
    save_to_h5py(h5file, output)

TypeError: Unsupported data type: <class 'jaxlib.xla_extension.ArrayImpl'>